In [1]:
import hashlib
from base64 import b64encode, b64decode
import pandas as pd
import qrcode as qr
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad


def encrypt_with_AES_CBC(secretKey, data, iv):
    try:
        keyBytes = hashlib.sha256(secretKey.encode('utf-8')).digest()
        cipher = AES.new(keyBytes, AES.MODE_CBC, iv.encode('utf-8'))
        cipherText = cipher.encrypt(pad(data.encode('utf-8'), AES.block_size))
        return b64encode(cipherText).decode('utf-8')
    except Exception as e:
        print(str(e))
        return None


def decrypt_with_AES_CBC(secretKey, data, iv):
    try:
        keyBytes = hashlib.sha256(secretKey.encode('utf-8')).digest()
        cipher = AES.new(keyBytes, AES.MODE_CBC, iv.encode('utf-8'))
        plainText = unpad(cipher.decrypt(b64decode(data.encode('utf-8'))), AES.block_size)
        return plainText.decode('utf-8').strip()
    except Exception as e:
        print(str(e))
        return None

In [7]:
file_path = r"D:\TA Materials\الفرق 2024\3.xlsx"
sheet_name = "Sheet1"
students = pd.read_excel(file_path, header=None, sheet_name=sheet_name)

In [8]:
students.head()

,0,1
0,إبراهيم محمود أحمد عادل عبدالبديع الشحات,1 CS
1,أحمد أشرف أحمد العربي محمود,1 CS
2,أحمد أيمن فتحي محمد إبراهيم العيسوي,1 CS
3,أحمد إيهاب صاحي حسين مرسي,1 CS
4,أحمد السيد عبدالله السيد كيلاني,1 CS


In [9]:
students.tail()

,0,1
742,محمود أحمد السيد عبدالعزيز,NaN
743,احمد نصر محمد سليمان,NaN
744,محمد احمد جوده عبدالرافع,NaN
745,رنا ممدوح الدسوقي مرسي,NaN
746,عبير خالد عبد الجواد ابراهيم,NaN


### Add new student to the dataframe

In [10]:
# add new student name to the dataframe
def add_student(name):
    new_data = {0: name}
    return students.append(new_data, ignore_index=True)

In [11]:
students = add_student("امير مأمون عبد الرحمن امين")

In [12]:
students.tail()

,0,1
743,احمد نصر محمد سليمان,NaN
744,محمد احمد جوده عبدالرافع,NaN
745,رنا ممدوح الدسوقي مرسي,NaN
746,عبير خالد عبد الجواد ابراهيم,NaN
747,امير مأمون عبد الرحمن امين,NaN


### Create Qr code for the new students

In [23]:
# create a qr code of the name of the student and his roll number and save it in the folder
for i in range(students.shape[0]):
    qr_data = str(students.iloc[i, 0]) + "\n" + str(i + 2)
    secretKey = "Secretkey"  # replace with your secret key
    iv = "##########"  # replace with your iv
    qr_data = encrypt_with_AES_CBC(secretKey, qr_data, iv)
    qr.make(qr_data).save(r"D:\QR Codes\QR" + str(i) + ".png")
    # qr.make(qr_data).save(r"D:\QR Codes\\" + str(students.iloc[i,0]) + str(students.iloc[i,1])+".png")


In [24]:
# add qr code to the dataframe 
students.head()

,0,1
0,إبراهيم محمود أحمد عادل عبدالبديع الشحات,1 CS
1,أحمد أشرف أحمد العربي محمود,1 CS
2,أحمد أيمن فتحي محمد إبراهيم العيسوي,1 CS
3,أحمد إيهاب صاحي حسين مرسي,1 CS
4,أحمد السيد عبدالله السيد كيلاني,1 CS


In [25]:
writer = pd.ExcelWriter('Students.xlsx', engine='xlsxwriter')
students.to_excel(writer, sheet_name='Sheet1', index=False)

In [7]:
from PIL import Image

workbook = writer.book
my_format = workbook.add_format()
my_format.set_align('vcenter')

worksheet = writer.sheets['Sheet1']
worksheet.set_default_row(44)

default_column_width = 8.43

for i in range(students.shape[0]):
    worksheet.write('A' + str(i + 2), "ـ" + students.iloc[i, 1] + ":-" + students.iloc[i, 0], my_format)
    image_path = r"D:\QR Codes\QR" + str(i) + ".png"
    # Get the dimensions of the image
    with Image.open(image_path) as img:
        image_width, image_height = img.size

    # Calculate the scale factors
    x_scale = 55 / image_width
    y_scale = 55 / image_height

    # Calculate the scaled image width and height
    scaled_image_width = image_width * x_scale
    scaled_image_height = image_height * y_scale

    # Calculate the offsets
    x_offset = int((60 - scaled_image_width) / 2)
    y_offset = int((60 - scaled_image_height) / 2)

    # Insert the image
    worksheet.set_column('B:B', None, my_format)
    worksheet.insert_image('B' + str(i + 2), r"D:\QR Codes\QR" + str(i) + ".png",
                           {'x_scale': x_scale, 'y_scale': y_scale, 'x_offset': x_offset, 'y_offset': y_offset})


In [ ]:
writer.save()

## Output

<img src="img.png"/>